In [1]:
!ls

0. Main program.txt
0. Source code_Walkthrough_Delivery1&2.ipynb
0. Source code_Walkthrough_Delivery3.ipynb
0. Source code_Walkthrough_Delivery4.ipynb
1. Data_Preparation_V1a_to debug.ipynb
1. Data_Preparation_V1b_to debug.ipynb
1.A Data_Preparation.py
1.B Data_Preparation.py
2.A Data Understanding_EDA.py
2.A Data_Understanding_EDA_1.0.ipynb
2.A.1 Dual dashboard.py
SIR_modeling_2022_exercise.ipynb
reference help


In [2]:
import pandas as pd
import numpy as np

from datetime import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Data Preparation

* focus is always to understand the final data structure
* support each step by visual analytics

## Data reference
### Johns Hopkins GITHUB csv data

https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
|-> Infections

### Centers for Civic Impact

https://github.com/govex/COVID-19/blob/a96dbc70eada30e83b1c475a328bb3cab4712741/data_tables/vaccine_data/global_data/time_series_covid19_vaccine_global.csv |-> Vaccinned

## 1 Get data and prepare delivery 4

In [ ]:
# %load /Users/victhorvic/ads_covid-19/src/data/get_data.py
#!/usr/bin/env python3
"""
Created on Mon Jul  4 18:59:23 2022

@author: victhorvic
"""
# %% [0] Libraries
import pandas as pd
import numpy as np

from datetime import datetime


import matplotlib.pyplot as plt

#%% [1] Get data

'''
Links of the data

Johns Hopkins GITHUB csv data
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series |-> Infections

Centers for Civic Impact
https://github.com/govex/COVID-19/blob/a96dbc70eada30e83b1c475a328bb3cab4712741/data_tables/vaccine_data/global_data/time_series_covid19_vaccine_global.csv |-> Vaccinned
'''

data_path = 'https://github.com/CSSEGISandData/COVID-19/blob/246eab67395dce9a4238fff77aa5f3561e253d48/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
## added ?raw=true to get last value and get the permantlink from Github

data_path1 = 'https://github.com/govex/COVID-19/blob/a96dbc70eada30e83b1c475a328bb3cab4712741/data_tables/vaccine_data/global_data/time_series_covid19_vaccine_global.csv?raw=true'
## added ?raw=true to get last value and get the permantlink from Github


# data infections 
pd_raw_inf=pd.read_csv(data_path,)

# data for vaccined
pd_raw_vac=pd.read_csv(data_path1,)

# %% [2] Vaccination data
# to prepare the index timer 
time_idx=pd_raw_vac['Date'].unique()

df_plot = pd.DataFrame({
    'Date':time_idx})


pd2 = pd_raw_vac.drop(['Report_Date_String','UID','Province_State','People_partially_vaccinated'],axis=1)
''' check data of the countries '''

#Verify Germany data 
testde = pd_raw_vac[pd_raw_vac['Country_Region']=='Germany'].iloc[:,1::].set_index(['Date'])
#testde.dropna(subset=['People_fully_vaccinated']).iloc[:,0:3].plot(title = 'Germany')

#US Data
#pd2[pd2['Country_Region']=='US'].iloc[:,1::].set_index('Date')
#pd2[pd2['Country_Region']=='US'].iloc[:,1::].set_index('Date').plot(title = 'US')

# Mexico data
#pd2[pd2['Country_Region']=='Mexico'].iloc[:,1::].set_index('Date')
#pd2[pd2['Country_Region']=='Mexico'].iloc[:,1::].set_index('Date').plot(title ='Mexico')

# %%% [2.1] Cocatenate data
'''Cocatenate data '''

country_list=['Mexico',
              'US',
              'Germany'
             ]

# define the data for concatenate
c_de = testde.dropna(subset=['People_fully_vaccinated']).iloc[:,2:3]

c_mx =pd2[pd2['Country_Region'] == country_list[0]].groupby('Date').sum().drop(['Doses_admin'], axis=1)
                             
c_us =pd2[pd2['Country_Region'] == country_list[1]].iloc[:,1::].groupby('Date').sum().drop(['Doses_admin'], axis=1)                             

#concac =pd.concat([c_mx,c_us,c_de],axis=1,join="inner")
concac =pd.concat([c_mx,c_us,c_de],axis=1,join="inner", )#keys=['Mexico', 'US', 'Germany'])
concac.columns = country_list

# %%%% [2.2] Verify data plot
# Total population for each country
        # Mexico US Germany
pop = [128.9E6, 329.5E6, 83.24E6]

''' Look for divide the data # not used '''
#concac[( 'Mexico')] = concac[('Mexico')].div(pop[0])
#concac[( 'US')] = concac[('US')].div(pop[1])
#concac[( 'Germany')] = concac[('Germany')].div(pop[2])


#concac[( 'Mexico', 'People_fully_vaccinated')] = concac[( 'Mexico', 'People_fully_vaccinated')].div(pop[0])
#concac[( 'US', 'People_fully_vaccinated')] = concac[( 'US', 'People_fully_vaccinated')].div(pop[1])
#concac[( 'Germany', 'People_fully_vaccinated')] = concac[( 'Germany', 'People_fully_vaccinated')].div(pop[2])

#not process data
concac[( 'Mexico')] = concac[('Mexico')]
concac[( 'US')] = concac[('US')]
concac[( 'Germany')] = concac[('Germany')]

# test the value 
#concac.plot()

# %%% [2.3] Save fig graph

''' Figure2: The vaccination rate over time '''

fig2 =concac.plot( grid = True,
                  xlabel = 'Date',
                  ylabel = 'Rate of the population',
                  title = 'The vaccination rate over time',  
                  
                  )

''' Save fig in the folder '''
fig2.figure.savefig('/Users/victhorvic/ads_covid-19/reports/figures/figure_2.png',dpi = 1000)
plt.show()

# %%% [2.4] Save to csv
'''Export csv file'''
df_plot1 = concac.reset_index()
df_plot1.to_csv('/Users/victhorvic/ads_covid-19/data/processed/COVID_vac_small_flat_table1.csv',sep=';',index=False)

# %% [3.1] Infections data
'''
Data for infections COVID¶
'''

# define time index
time_idx=pd_raw_inf.columns[4:]

df_plot= pd.DataFrame({'date':time_idx})

# define countries
country_list=['Mexico',
              'US',
              'Germany',
             ] 

for each in country_list:
    df_plot[each] = np.array(pd_raw_inf[pd_raw_inf['Country/Region']==each].iloc[:,4::].sum(axis=0))

#to see the graph
#df_plot.set_index('date').plot()


# %%% [3.2] Save fig graph
# Total population for each country: 
    # Mexico US Germany
pop = [128.9E6, 329.5E6, 83.24E6]

fig1 =df_plot.set_index('date').div(pop).plot( grid = True,
                                              xlabel = 'Date',
                                              ylabel = 'Confirmed cases (absolute Covid cases/population size)',
                                              title = 'The relative cases overtime of Covid infectors',        
                                              )

''' Save fig in the folder '''
#fig11 = fig1.figure 
#fig11.savefig('figure_1.png', dpi=300,)
fig1.figure.savefig('/Users/victhorvic/ads_covid-19/reports/figures/figure_1.png', dpi = 1000)
plt.show()

#to divided the data over population
#df_plot = df_plot.set_index('date').div(pop)
# not divided data 
df_plot = df_plot.set_index('date')
df_plot = df_plot.reset_index()

# %%% [3.3] Save to csv
'''Export csv file'''


''' Fix time_index'''
time_idx=[datetime.strptime( each,"%m/%d/%y") for each in df_plot.date] 
# convert to datetime

time_str=[each.strftime('%Y-%m-%d') for each in time_idx] 
# convert back to date ISO norm (str)


df_plot['date']=time_idx
type(df_plot['date'][0])


df_plot.to_csv('/Users/victhorvic/ads_covid-19/data/processed/COVID_inf_small_flat_table1.csv',sep=';',index=False)




## 2 Models 
### 2.1 Predict Model Germany

In [ ]:
# %load /Users/victhorvic/ads_covid-19/src/models/predict_model_Germany.py
#!/usr/bin/env python3
"""
Created on Mon Jul  4 18:59:23 2022

@author: victhorvic
"""
# %% [0] Libraries
import pandas as pd
import numpy as np


from scipy import optimize
from scipy import integrate

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

sns.set(style="darkgrid")

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

# %% [1] Get data

# try to parse the dates right at the beginning 
# it works out of the box if the date was stored ISO YYYY-MM-DD format

# infections
df_analyse = pd.read_csv('/Users/victhorvic/ads_covid-19/data/processed/COVID_inf_small_flat_table1.csv',
                           sep=';', parse_dates=[0])

# %% [2] Basic parameters
# set some basic parameters
# beta/gamma is denoted as  'basic reproduction number'

N0=1000000 #max susceptible population
beta=0.4  # infection spread dynamics
gamma= 0.1  # recovery rate


# condition I0+S0+R0=N0
I0=df_analyse.Germany[8]
S0=N0-I0
R0=0

# %%% [2.1] SIR model
def SIR_model(SIR,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return([dS_dt,dI_dt,dR_dt])


# %% [3] Simulative SIR curves
'''Simulative approach to calculate SIR curves'''
SIR=np.array([S0,I0,R0])
propagation_rates=pd.DataFrame(columns={'susceptible':S0,
                                        'infected':I0,
                                        'recovered':R0})



for each_t in np.arange(877-8):
   
    new_delta_vec=SIR_model(SIR,beta,gamma)
   
    SIR=SIR+new_delta_vec
    
    propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                'infected':SIR[1],
                                                'recovered':SIR[2]}, ignore_index=True);

# %%% [3.1] Plot Simulative
fig, ax1 = plt.subplots(1, 1)

ax1.plot(propagation_rates.index,propagation_rates.infected,label='infected',color='k')
ax1.plot(propagation_rates.index,propagation_rates.recovered,label='recovered')
ax1.plot(propagation_rates.index,propagation_rates.susceptible,label='susceptible')

ax1.set_ylim(10, 1500000)
ax1.set_yscale('linear')
ax1.set_title('Szenario SIR simulations (demonstration purposes only)',size=16)
ax1.set_xlabel('time in days',size=16)
ax1.legend(loc='best',
           prop={'size': 16});


# %% [4] Fitting the parameters of SIR model

ydata = np.array(df_analyse.Germany[65:])
t=np.arange(len(ydata))

# ensure re-initialization 
I0=ydata[0]
S0=N0-I0
R0=0
#beta _ beta valuee?

# %%% Functions SIR_model_t , integration
def SIR_model_t(SIR,t,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return dS_dt,dI_dt,dR_dt

def fit_odeint(x, beta, gamma):
    '''
    helper function for the integration
    '''
    return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1] # we only would like to get dI

# %%% Curve of the dif eq
# example curve of our differential equationa
popt=[0.06,-0.004]
curve = fit_odeint(t, *popt)
# the resulting curve has to be fitted
# free parameters are here beta and gamma

fig = plt.figure()
plt.plot(curve)

# %%% Optimization
popt, pcov = optimize.curve_fit(fit_odeint, t, ydata)
perr = np.sqrt(np.diag(pcov))
print('\n\n\n\n ')    
print('standard deviation errors : ',str(perr), ' start infect:',ydata[0])
print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])

# get the final fitted curve
fitted=fit_odeint(t, *popt)

fig1 = plt.figure()
plt.plot(fitted)

# %% Graph
fig1 = plt.figure()
plt.semilogy(t, ydata, 'o')
plt.semilogy(t, fitted)
plt.title("Fit of SIR model for Germany cases")
plt.ylabel("Population infected")
plt.xlabel("Days")
plt.show()
print('\n\n\n')  
print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])
print("Basic Reproduction Number R0 " , popt[0]/ popt[1])
print("This ratio is derived as the expected number of new infections (these new infections are sometimes called secondary infections from a single infection in a population where all subjects are susceptible. @wiki")

# %%% Export to CSV
''' Export to CSV'''

Table = pd.DataFrame([t,ydata, fitted]).T
Table.columns = ('t', 'ydata', 'fitted')
Table.to_csv('/Users/victhorvic/ads_covid-19/data/interim/COVID_PR_Ger_table1.csv',
                         sep=';',index=False)










# %%% Export to optimum value 

np.savetxt('/Users/victhorvic/ads_covid-19/data/interim/popt_Germany.txt', popt)








# %% Dynamic beta in SIR (infection rate)

t_initial=50
t_intro_measures=300
t_hold=310
t_relax=210

beta_max = 2
beta_min = -0.0011
gamma = - 0.0043
pd_beta=np.concatenate((np.array(t_initial*[beta_max]),
                       np.linspace(beta_max,beta_min,t_intro_measures),
                       np.array(t_hold*[beta_min]),
                        np.linspace(beta_min,beta_max,t_relax),
                       ))


#pd_beta

SIR=np.array([S0,I0,R0])
propagation_rates=pd.DataFrame(columns={'susceptible':S0,
                                        'infected':I0,
                                        'recovered':R0})

for each_beta in pd_beta:
   
    new_delta_vec=SIR_model(SIR,each_beta,gamma)
   
    SIR=SIR+new_delta_vec
    
    propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                'infected':SIR[1],
                                                'recovered':SIR[2]}, ignore_index=True)
# %%
# Plot

plt.figure()
fig2, ax2 = plt.subplots(1, 1)

#ax2.plot(propagation_rates.index,propagation_rates.infected,label='infected',linewidth=3)

t_phases=np.array([t_initial,t_intro_measures,t_hold,t_relax]).cumsum()

ax2.bar(np.arange(len(ydata)),ydata, width=0.5,label='current infected Germany', color='r')
#ax2.axvspan(0,t_phases[0], facecolor='b', alpha=0.2,label='no measures')
#ax2.axvspan(t_phases[0],t_phases[1], facecolor='b', alpha=0.3,label='hard measures introduced')
#ax2.axvspan(t_phases[1],t_phases[2], facecolor='b', alpha=0.4,label='hold measures')
#ax2.axvspan(t_phases[2],t_phases[3], facecolor='b', alpha=0.5,label='relax measures')
#ax2.axvspan(t_phases[3],len(propagation_rates.infected), facecolor='b', alpha=0.6,label='repead hard measures')

ax2.set_ylim(10, 1.5*max(propagation_rates.infected))
ax2.set_yscale('log')
ax2.set_title('Szenario SIR simulations  (demonstration purposes only)',size=16)
ax2.set_xlabel('time in days',size=16)
#ax2.legend(loc='best',
#           prop={'size': 16});


### 2.2 Predict Model Mexico

In [ ]:
# %load /Users/victhorvic/ads_covid-19/src/models/predict_model_Mexico.py
#!/usr/bin/env python3
"""
Created on Fri Jul  8 18:56:16 2022

@author: victhorvic
"""

# %% [0] Libraries
import pandas as pd
import numpy as np


from scipy import optimize
from scipy import integrate

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

sns.set(style="darkgrid")

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

# %% [0] Funtions

# %%% [2.1] SIR model
def SIR_model(SIR,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return([dS_dt,dI_dt,dR_dt])


# %%% Functions SIR_model_t , integration
def SIR_model_t(SIR,t,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return dS_dt,dI_dt,dR_dt

def fit_odeint(x, beta, gamma):
    '''
    helper function for the integration
    '''
    return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1] # we only would like to get dI


# %% [1] Get data

# try to parse the dates right at the beginning 
# it works out of the box if the date was stored ISO YYYY-MM-DD format

# infections
df_analyse = pd.read_csv('/Users/victhorvic/ads_covid-19/data/processed/COVID_inf_small_flat_table1.csv',
                           sep=';', parse_dates=[0])

# %% [2] Basic parameters
# set some basic parameters
# beta/gamma is denoted as  'basic reproduction number'

N0=1000000 #max susceptible population
beta=0.4  # infection spread dynamics
gamma= 0.1  # recovery rate


# condition I0+S0+R0=N0
I0=df_analyse.Mexico[60] 
S0=N0-I0
R0=0



# %% [3] Simulative SIR curves
'''Simulative approach to calculate SIR curves'''
SIR=np.array([S0,I0,R0])
propagation_rates=pd.DataFrame(columns={'susceptible':S0,
                                        'infected':I0,
                                        'recovered':R0})



for each_t in np.arange(len(df_analyse.Mexico)-65):
   
    new_delta_vec=SIR_model(SIR,beta,gamma)
   
    SIR=SIR+new_delta_vec
    
    propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                'infected':SIR[1],
                                                'recovered':SIR[2]}, ignore_index=True);

# %%% [3.1] Plot Simulative
fig, ax1 = plt.subplots(1, 1)

ax1.plot(propagation_rates.index,propagation_rates.infected,label='infected',color='k')
ax1.plot(propagation_rates.index,propagation_rates.recovered,label='recovered')
ax1.plot(propagation_rates.index,propagation_rates.susceptible,label='susceptible')

ax1.set_ylim(10, 1500000)
ax1.set_yscale('linear')
ax1.set_title('Szenario SIR simulations (demonstration purposes only)',size=16)
ax1.set_xlabel('time in days',size=16)
ax1.legend(loc='best',
           prop={'size': 16});


# %% [4] Fitting the parameters of SIR model

ydata = np.array(df_analyse.Mexico[65:])
t=np.arange(len(ydata))

# ensure re-initialization 
I0=ydata[0]
S0=N0-I0
R0=0
#beta _ beta valuee?


# %%% Curve of the dif eq
# example curve of our differential equationa
popt=[0.4,0.1]
curve = fit_odeint(t, *popt)
# the resulting curve has to be fitted
# free parameters are here beta and gamma

fig = plt.figure()
plt.plot(curve)

# %%% Optimization
popt, pcov = optimize.curve_fit(fit_odeint, t, ydata)
perr = np.sqrt(np.diag(pcov))
print('\n\n\n\n ')    
print('standard deviation errors : ',str(perr), ' start infect:',ydata[0])
print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])

# get the final fitted curve
fitted=fit_odeint(t, *popt)

fig1 = plt.figure()
plt.plot(fitted)

# %% Graph
fig1 = plt.figure()
plt.semilogy(t, ydata, 'o')
plt.semilogy(t, fitted)
plt.title("Fit of SIR model for Germany cases")
plt.ylabel("Population infected")
plt.xlabel("Days")
plt.show()
print('\n\n\n')  
print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])
print("Basic Reproduction Number R0 " , popt[0]/ popt[1])
print("This ratio is derived as the expected number of new infections (these new infections are sometimes called secondary infections from a single infection in a population where all subjects are susceptible. @wiki")

# %%% Export to CSV
''' Export to CSV'''

Table = pd.DataFrame([t,ydata, fitted]).T
Table.columns = ('t', 'ydata', 'fitted')
Table.to_csv('/Users/victhorvic/ads_covid-19/data/interim/COVID_PR_Mx_table1.csv',
                         sep=';',index=False)

# %%% Export to optimum value 

np.savetxt('/Users/victhorvic/ads_covid-19/data/interim/popt_Mx.txt', popt)

# %% Dynamic beta in SIR (infection rate)
'''
t_initial=50
t_intro_measures=300
t_hold=310
t_relax=210

beta_max = 2
beta_min = -0.0011
gamma = - 0.0043
pd_beta=np.concatenate((np.array(t_initial*[beta_max]),
                       np.linspace(beta_max,beta_min,t_intro_measures),
                       np.array(t_hold*[beta_min]),
                        np.linspace(beta_min,beta_max,t_relax),
                       ))


#pd_beta

SIR=np.array([S0,I0,R0])
propagation_rates=pd.DataFrame(columns={'susceptible':S0,
                                        'infected':I0,
                                        'recovered':R0})

for each_beta in pd_beta:
   
    new_delta_vec=SIR_model(SIR,each_beta,gamma)
   
    SIR=SIR+new_delta_vec
    
    propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                'infected':SIR[1],
                                                'recovered':SIR[2]}, ignore_index=True)
# %%
# Plot

plt.figure()
fig2, ax2 = plt.subplots(1, 1)

#ax2.plot(propagation_rates.index,propagation_rates.infected,label='infected',linewidth=3)

t_phases=np.array([t_initial,t_intro_measures,t_hold,t_relax]).cumsum()

ax2.bar(np.arange(len(ydata)),ydata, width=0.5,label='current infected Germany', color='r')
#ax2.axvspan(0,t_phases[0], facecolor='b', alpha=0.2,label='no measures')
#ax2.axvspan(t_phases[0],t_phases[1], facecolor='b', alpha=0.3,label='hard measures introduced')
#ax2.axvspan(t_phases[1],t_phases[2], facecolor='b', alpha=0.4,label='hold measures')
#ax2.axvspan(t_phases[2],t_phases[3], facecolor='b', alpha=0.5,label='relax measures')
#ax2.axvspan(t_phases[3],len(propagation_rates.infected), facecolor='b', alpha=0.6,label='repead hard measures')

ax2.set_ylim(10, 1.5*max(propagation_rates.infected))
ax2.set_yscale('log')
ax2.set_title('Szenario SIR simulations  (demonstration purposes only)',size=16)
ax2.set_xlabel('time in days',size=16)
#ax2.legend(loc='best',
#           prop={'size': 16});
'''

### 2.2 Predict Model US

In [ ]:
# %load /Users/victhorvic/ads_covid-19/src/models/predict_model_US.py
#!/usr/bin/env python3
"""
Created on Fri Jul  8 18:56:16 2022

@author: victhorvic
"""

# %% [0] Libraries
import pandas as pd
import numpy as np


from scipy import optimize
from scipy import integrate

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

sns.set(style="darkgrid")

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

# %% [0] Funtions

# %%% [2.1] SIR model
def SIR_model(SIR,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return([dS_dt,dI_dt,dR_dt])


# %%% Functions SIR_model_t , integration
def SIR_model_t(SIR,t,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return dS_dt,dI_dt,dR_dt

def fit_odeint(x, beta, gamma):
    '''
    helper function for the integration
    '''
    return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1] # we only would like to get dI


# %% [1] Get data

# try to parse the dates right at the beginning 
# it works out of the box if the date was stored ISO YYYY-MM-DD format

# infections
df_analyse = pd.read_csv('/Users/victhorvic/ads_covid-19/data/processed/COVID_inf_small_flat_table1.csv',
                           sep=';', parse_dates=[0])

# %% [2] Basic parameters
# set some basic parameters
# beta/gamma is denoted as  'basic reproduction number'

N0=10000000 #max susceptible population
beta=0.4  # infection spread dynamics
gamma= 0.1  # recovery rate


# condition I0+S0+R0=N0
I0=df_analyse.US[30]
S0=N0-I0
R0=0



# %% [3] Simulative SIR curves
'''Simulative approach to calculate SIR curves'''
SIR=np.array([S0,I0,R0])
propagation_rates=pd.DataFrame(columns={'susceptible':S0,
                                        'infected':I0,
                                        'recovered':R0})



for each_t in np.arange(len(df_analyse.US)-15):
   
    new_delta_vec=SIR_model(SIR,beta,gamma)
   
    SIR=SIR+new_delta_vec
    
    propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                'infected':SIR[1],
                                                'recovered':SIR[2]}, ignore_index=True);

# %%% [3.1] Plot Simulative
fig, ax1 = plt.subplots(1, 1)

ax1.plot(propagation_rates.index,propagation_rates.infected,label='infected',color='k')
ax1.plot(propagation_rates.index,propagation_rates.recovered,label='recovered')
ax1.plot(propagation_rates.index,propagation_rates.susceptible,label='susceptible')

ax1.set_ylim(10, 100000000)
ax1.set_yscale('linear')
ax1.set_title('Szenario SIR simulations (demonstration purposes only)',size=16)
ax1.set_xlabel('time in days',size=16)
ax1.legend(loc='best',
           prop={'size': 16});


# %% [4] Fitting the parameters of SIR model

ydata = np.array(df_analyse.US[65:])
t=np.arange(len(ydata))

# ensure re-initialization 
I0=ydata[0]
S0=N0-I0
R0=0
#beta _ beta valuee?


# %%% Curve of the dif eq
# example curve of our differential equationa
popt=[0.4,0.1]
curve = fit_odeint(t, *popt)
# the resulting curve has to be fitted
# free parameters are here beta and gamma

fig = plt.figure()
plt.plot(curve)

# %%% Optimization
popt, pcov = optimize.curve_fit(fit_odeint, t, ydata)
perr = np.sqrt(np.diag(pcov))
print('\n\n\n\n ')    
print('standard deviation errors : ',str(perr), ' start infect:',ydata[0])
print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])

# get the final fitted curve
fitted=fit_odeint(t, *popt)

fig1 = plt.figure()
plt.plot(fitted)

# %% Graph
fig1 = plt.figure()
plt.semilogy(t, ydata, 'o')
plt.semilogy(t, fitted)
plt.title("Fit of SIR model for Germany cases")
plt.ylabel("Population infected")
plt.xlabel("Days")
plt.show()
print('\n\n\n')  
print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])
print("Basic Reproduction Number R0 " , popt[0]/ popt[1])
print("This ratio is derived as the expected number of new infections (these new infections are sometimes called secondary infections from a single infection in a population where all subjects are susceptible. @wiki")

# %%% Export to CSV
''' Export to CSV'''

Table = pd.DataFrame([t,ydata, fitted]).T
Table.columns = ('t', 'ydata', 'fitted')
Table.to_csv('/Users/victhorvic/ads_covid-19/data/interim/COVID_PR_US_table1.csv',
                         sep=';',index=False)

# %%% Export to optimum value 

np.savetxt('/Users/victhorvic/ads_covid-19/data/interim/popt_US.txt', popt)

# %% Dynamic beta in SIR (infection rate)
'''
t_initial=50
t_intro_measures=300
t_hold=310
t_relax=210

beta_max = 2
beta_min = -0.0011
gamma = - 0.0043
pd_beta=np.concatenate((np.array(t_initial*[beta_max]),
                       np.linspace(beta_max,beta_min,t_intro_measures),
                       np.array(t_hold*[beta_min]),
                        np.linspace(beta_min,beta_max,t_relax),
                       ))


#pd_beta

SIR=np.array([S0,I0,R0])
propagation_rates=pd.DataFrame(columns={'susceptible':S0,
                                        'infected':I0,
                                        'recovered':R0})

for each_beta in pd_beta:
   
    new_delta_vec=SIR_model(SIR,each_beta,gamma)
   
    SIR=SIR+new_delta_vec
    
    propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                'infected':SIR[1],
                                                'recovered':SIR[2]}, ignore_index=True)
# %%
# Plot

plt.figure()
fig2, ax2 = plt.subplots(1, 1)

#ax2.plot(propagation_rates.index,propagation_rates.infected,label='infected',linewidth=3)

t_phases=np.array([t_initial,t_intro_measures,t_hold,t_relax]).cumsum()

ax2.bar(np.arange(len(ydata)),ydata, width=0.5,label='current infected Germany', color='r')
#ax2.axvspan(0,t_phases[0], facecolor='b', alpha=0.2,label='no measures')
#ax2.axvspan(t_phases[0],t_phases[1], facecolor='b', alpha=0.3,label='hard measures introduced')
#ax2.axvspan(t_phases[1],t_phases[2], facecolor='b', alpha=0.4,label='hold measures')
#ax2.axvspan(t_phases[2],t_phases[3], facecolor='b', alpha=0.5,label='relax measures')
#ax2.axvspan(t_phases[3],len(propagation_rates.infected), facecolor='b', alpha=0.6,label='repead hard measures')

ax2.set_ylim(10, 1.5*max(propagation_rates.infected))
ax2.set_yscale('log')
ax2.set_title('Szenario SIR simulations  (demonstration purposes only)',size=16)
ax2.set_xlabel('time in days',size=16)
#ax2.legend(loc='best',
#           prop={'size': 16});
'''

## 3 Visualization

Visualize data on dashboard 

In [ ]:
# %load /Users/victhorvic/ads_covid-19/src/visualization/visualize_Delivery 4.py
#!/usr/bin/env python3
"""
Created on Sat Jul 23 17:00:19 2022

@author: victhorvic
"""


# %% [0] Libraries 
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
print(os.getcwd())

# %% [1] data for Model

#Data from build_features_2.py
df_input_large=pd.read_csv('/Users/victhorvic/ads_covid-19/data/processed/COVID_final_set_2.csv',sep=';')

countries = df_input_large.columns[1:200]

# %%% [1] data for Model SIR

pd_model_ger = pd.read_csv('/Users/victhorvic/ads_covid-19/data/interim/COVID_PR_Ger_table1.csv',
                         sep=';',parse_dates=[0]).drop(['t'], axis = 1)

pd_model_mx = pd.read_csv('/Users/victhorvic/ads_covid-19/data/interim/COVID_PR_Mx_table1.csv',
                         sep=';',parse_dates=[0]).drop(['t'], axis = 1)

pd_model_us = pd.read_csv('/Users/victhorvic/ads_covid-19/data/interim/COVID_PR_US_table1.csv',
                         sep=';',parse_dates=[0]).drop(['t'], axis = 1)



''' Concatenate '''
pd_plot1 =pd.concat([pd_model_ger,pd_model_mx,pd_model_us],
                    axis = 'columns',join="inner", keys=['Germany', 'Mexico', 'US'], 
                    names=['Country'])

# %%% [1.5] Import the optimum value 
''' Beta [1] Gamma [2] '''
popt_Mx = np.loadtxt('/Users/victhorvic/ads_covid-19/data/interim/popt_Mx.txt')
popt_US = np.loadtxt('/Users/victhorvic/ads_covid-19/data/interim/popt_US.txt')
popt_Ge = np.loadtxt('/Users/victhorvic/ads_covid-19/data/interim/popt_Germany.txt')


# %% [2] Dash app

'''To refererence links:
    https://medium.com/analytics-vidhya/plotting-multiple-figures-with-live-data-using-dash-and-plotly-4f5277870cd7

    https://plotly.com/python/subplots/
    https://community.plotly.com/t/two-graphs-side-by-side/5312/2
'''
fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data

    Goal of the project is to teach data science by applying a cross industry standard process,
    it covers the full walkthrough of: automated data gathering, data transformations,
    filtering and machine learning to approximating the doubling time, and
    (static) deployment of responsive dashboard.

    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in countries],
        value=['US', 'Germany','Mexico'], # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),


    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed ', 'value': 1},
        {'label': 'Timeline Confirmed Filtered', 'value': 2},
        {'label': 'Timeline Doubling Rate', 'value': 3},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 4},
    ],
    value=1,
    multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope'),
    
    # SIR Model Section
    html.Div([
        html.H3('Fit of SIR model for country cases'),
        html.Label('Multi-Select Country'),
    
        dcc.Dropdown(
            id='country_drop_down2',
            options=[
                {'label': 'Mexico', 'value': 'Mexico'},
                {'label': 'US', 'value': 'US'},
                {'label': 'Germany', 'value': 'Germany'},

        ],
            value=[ 'Germany', 'US', 'Mexico'], # which are pre-selected
            multi= True #_ boolean value to show diferent countries values
            
            ),   
        
        dcc.Graph(figure=fig, id='main_window_slope2'),
        
        dcc.Markdown('''### Optimal parameters per country:'''),        
        html.Footer(' Germany = beta {:.5f} and gamma {:.5f} \n'.format(popt_Ge[0],popt_Ge[1])),
        html.Footer('US      = beta {:.5f} and gamma {:.5f} \n'.format(popt_US[0],popt_US[1])),
        html.Footer('Mexico  = beta {:.5f} and gamma {:.5f} \n'.format(popt_Mx[0],popt_Mx[1])),
        html.Footer('\rThis ratio is derived as the expected number of new infections (these new infections are sometimes called secondary infections from a single infection in a population where all subjects are susceptible. @wiki'),
        html.Footer('''Data from: 
        Johns Hopkins GITHUB csv data
        https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series |-> Infections '''),
        #html.Footer('''Centers for Civic Impact https://github.com/govex/COVID-19/blob/a96dbc70eada30e83b1c475a328bb3cab4712741/data_tables/vaccine_data/global_data/time_series_covid19_vaccine_global.csv |-> Vaccinned '''),
    ]),
    
       
])

# %% [3] Callback function 

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
def update_figure(country_list,show_doubling):


    if show_doubling > 2:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
              }


    traces = []
    for each in country_list:
        
        if show_doubling == 2: 
            df_plot=df_input_large[each+'_filter']
        elif show_doubling == 3: 
            df_plot=df_input_large[each+'_DR']
        elif show_doubling == 4: 
            df_plot=df_input_large[each+'_filter_DR']
        else:
            df_plot=df_input_large[each]

        traces.append(dict(x=df_input_large.date,
                                y=df_plot,
                                mode='markers+lines',
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }

# %% [3.1] Callback SIR

''' SIR Dashboard'''
@app.callback(
    Output('main_window_slope2', 'figure'),
    [Input('country_drop_down2', 'value')])
def update_figure2(country_list):
  
    traces = [] 
 
    '''  fig.add_trace(go.Scatter(#x=pd_plot1.index,
                                    y=pd_plot1.Germany.ydata,
                                    mode='markers+lines',
                                    opacity=0.9,
                                    line_width=2,
                                    marker_size=4, 
                                    name='Germany real data',
                                    
                                     )

    
    '''
    for each in country_list:
        traces.append(dict( #x=df_plot1_i.date,
                                y=pd_plot1[each, 'ydata'],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4, 
                                name= "Real data {}".format(each)
                        )
                )
   
        traces.append(dict( #x=df_plot1_i.date,
                                y=pd_plot1[each,'fitted'],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=3, 
                                name= "SIR data {}".format(each)   #SIR model data
                        )
                )
        
    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,
                xaxis_title="Days",
                yaxis_title="Population infected (source johns hopkins csse, log-scale)",
                xaxis={'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                        
                      },
                yaxis={'type':"log", #change to logaritmic scale #to change logaritmic "linear"
                       'range':'[1.1,8,8]'
                      },
                
                
        ),
            
    }
# %% [4] Run dashboard
app.run_server(debug=True, use_reloader=False)

